## Neural Part Of Speech Tagging

_This is an optional assignment. Turn back whilst thou still can_
_Это необязательное задание. Возвращайся, пока ты еще можешь_ 

We're now going to solve the same problem of POS tagging with neural networks.
<img src=https://i.stack.imgur.com/6pdIT.png width=320>

From deep learning perspective, this is a task of predicting a sequence of outputs aligned to a sequence of inputs. There are several problems that match this formulation:
* Part Of Speech Tagging -  an auxuliary task for many NLP problems
* Named Entity Recognition - for chat bots and web crawlers
* Protein structure prediction - for bioinformatics


С точки зрения глубокого обучения, это задача прогнозирования последовательности выходных данных, согласованных с последовательностью входных данных. Есть несколько проблем, которые соответствуют этой формулировке:

* Часть речевых тегов - вспомогательная задача для многих проблем НЛП
* Распознавание именованных сущностей - для чат-ботов и веб-сканеров
* Прогнозирование структуры белков - для биоинформатики 

In [1]:
%tensorflow_version 1.x
import nltk
import sys
import numpy as np

nltk.download('brown')
nltk.download('universal_tagset')
data = nltk.corpus.brown.tagged_sents(tagset='universal')
all_tags = ['#EOS#','#UNK#','ADV', 'NOUN', 'ADP', 'PRON', 'DET', '.', 'PRT', 'VERB', 'X', 'NUM', 'CONJ', 'ADJ']

data = np.array([ [(word.lower(),tag) for word,tag in sentence] for sentence in data ])

TensorFlow 1.x selected.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


In [2]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

In [3]:
from IPython.display import HTML, display
def draw(sentence):
    words, tags = zip(*sentence)
    display(HTML('<table><tr>{tags}</tr>{words}<tr></table>'.format(
                words = '<td>{}</td>'.format('</td><td>'.join(words)),
                tags = '<td>{}</td>'.format('</td><td>'.join(tags)))))
    
    
draw(data[11])
draw(data[10])
draw(data[7])

NOUN,ADP,NOUN,NOUN,NOUN,NOUN,VERB,ADV,VERB,ADP,DET,ADJ,NOUN,.,implementation,of,georgia's,automobile,title,law,was,also,recommended,by,the,outgoing,jury,.


PRON,VERB,ADP,DET,NOUN,.,VERB,NOUN,PRT,VERB,.,DET,NOUN,.,it,urged,that,the,city,``,take,steps,to,remedy,'',this,problem,.


NOUN,VERB,merger,proposed


### Building vocabularies

Just like before, we have to build a mapping from tokens to integer ids. This time around, our model operates on a word level, processing one word per RNN step. This means we'll have to deal with far larger vocabulary.

Luckily for us, we only receive those words as input i.e. we don't have to predict them. This means we can have a large vocabulary for free by using word embeddings.

### Создание словарей

Как и раньше, нам нужно построить сопоставление токенов с целочисленными идентификаторами. На этот раз наша модель работает на уровне слов, обрабатывая одно слово за шаг RNN. Это означает, что нам придется иметь дело с гораздо большим словарным запасом.

К счастью для нас, мы получаем только эти слова в качестве входных данных, т.е. нам не нужно их предсказывать. Это означает, что мы можем бесплатно иметь большой словарный запас, используя вложения слов. 

In [4]:
from collections import Counter
word_counts = Counter()
for sentence in data:
    words, tags = zip(*sentence)
    word_counts.update(words)

all_words = ['#EOS#','#UNK#'] + list(list(zip(*word_counts.most_common(10000)))[0])

#let's measure what fraction of data words are in the dictionary
# давайте измерим, какая часть слов данных находится в словаре 
print("Coverage = %.5f" % (float(sum(word_counts[w] for w in all_words)) / sum(word_counts.values())))

Coverage = 0.92876


In [5]:
from collections import defaultdict
word_to_id = defaultdict(lambda:1, { word: i for i, word in enumerate(all_words) })
tag_to_id = { tag: i for i, tag in enumerate(all_tags)}

convert words and tags into fixed-size matrix

In [6]:
def to_matrix(lines, token_to_id, max_len=None, pad=0, dtype='int32', time_major=False):
    """Converts a list of names into rnn-digestable matrix with paddings added after the end
       Преобразует список имен в удобоваримую матрицу с заполнением, добавленным после конца"""
    
    max_len = max_len or max(map(len,lines))
    matrix = np.empty([len(lines), max_len],dtype)
    matrix.fill(pad)

    for i in range(len(lines)):
        line_ix = list(map(token_to_id.__getitem__,lines[i]))[:max_len]
        matrix[i,:len(line_ix)] = line_ix

    return matrix.T if time_major else matrix



In [7]:
batch_words, batch_tags = zip(*[zip(*sentence) for sentence in data[-3:]])

print("Word ids:")
print(to_matrix(batch_words, word_to_id))
print("Tag ids:")
print(to_matrix(batch_tags, tag_to_id))

Word ids:
[[   2 3057    5    2 2238 1334 4238 2454    3    6   19   26 1070   69
     8 2088    6    3    1    3  266   65  342    2    1    3    2  315
     1    9   87  216 3322   69 1558    4    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  45   12    8  511 8419    6   60 3246   39    2    1    1    3    2
   845    1    3    1    3   10 9910    2    1 3470    9   43    1    1
     3    6    2 1046  385   73 4562    3    9    2    1    1 3250    3
    12   10    2  861 5240   12    8 8936  121    1    4]
 [  33   64   26   12  445    7 7346    9    8 3337    3    1 2811    3
     2  463  572    2    1    1 1649   12    1    4    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
Tag ids:
[[ 6  3  4  6  3  3  9  9  7 12  4  5  9  4  6  3 12  7  9  7  9  8  4  6
   3  7  6 13  3  4  6  3  9  4  3  7  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0

### Build model

Unlike our previous lab, this time we'll focus on a high-level keras interface to recurrent neural networks. It is as simple as you can get with RNN, allbeit somewhat constraining for complex tasks like seq2seq.

By default, all keras RNNs apply to a whole sequence of inputs and produce a sequence of hidden states `(return_sequences=True` or just the last hidden state `(return_sequences=False)`. All the recurrence is happening under the hood.

At the top of our model we need to apply a Dense layer to each time-step independently. As of now, by default keras.layers.Dense would apply once to all time-steps concatenated. We use __keras.layers.TimeDistributed__ to modify Dense layer so that it would apply across both batch and time axes.

### Построить модель

В отличие от нашей предыдущей лабораторной работы, на этот раз мы сосредоточимся на высокоуровневом интерфейсе keras для повторяющихся нейронных сетей. Это настолько просто, насколько вы можете получить с RNN, хотя и несколько ограничивает сложные задачи, такие как seq2seq.

По умолчанию все keras RNN применяются ко всей последовательности входных данных и создают последовательность скрытых состояний `(return_sequences = True` или только последнее скрытое состояние` (return_sequences = False) `. Все повторения происходят под капотом.

В верхней части нашей модели нам нужно применить плотный слой к каждому временному шагу независимо. На данный момент по умолчанию keras.layers.Dense будет применяться один раз ко всем объединенным временным шагам. Мы используем __keras.layers.TimeDistributed__ для изменения слоя Dense, чтобы он применялся как для пакетной, так и для временной осей. 

In [8]:
import keras
import keras.layers as L

model = keras.models.Sequential()
model.add(L.InputLayer([None],dtype='int32'))
model.add(L.Embedding(len(all_words),50))
model.add(L.SimpleRNN(64,return_sequences=True))

#add top layer that predicts tag probabilities
stepwise_dense = L.Dense(len(all_tags),activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


__Training:__ in this case we don't want to prepare the whole training dataset in advance. The main cause is that the length of every batch depends on the maximum sentence length within the batch. This leaves us two options: use custom training code as in previous seminar or use generators.

Keras models have a __`model.fit_generator`__ method that accepts a python generator yielding one batch at a time. But first we need to implement such generator:

__Training: __ в этом случае мы не хотим заранее готовить весь набор обучающих данных. Основная причина заключается в том, что длина каждого пакета зависит от максимальной длины предложения в пакете. Это оставляет нам два варианта: использовать специальный обучающий код, как на предыдущем семинаре, или использовать генераторы.

В моделях Keras есть метод __`model.fit_generator`__, который принимает генератор Python, выдающий по одной партии за раз. Но для начала нам нужно реализовать такой генератор: 

In [9]:
from keras.utils.np_utils import to_categorical
BATCH_SIZE=32
def generate_batches(sentences,batch_size=BATCH_SIZE,max_len=None,pad=0):
    assert isinstance(sentences,np.ndarray),"Make sure sentences is q numpy array"
    
    while True:
        indices = np.random.permutation(np.arange(len(sentences)))
        for start in range(0,len(indices)-1,batch_size):
            batch_indices = indices[start:start+batch_size]
            batch_words,batch_tags = [],[]
            for sent in sentences[batch_indices]:
                words,tags = zip(*sent)
                batch_words.append(words)
                batch_tags.append(tags)

            batch_words = to_matrix(batch_words,word_to_id,max_len,pad)
            batch_tags = to_matrix(batch_tags,tag_to_id,max_len,pad)

            batch_tags_1hot = to_categorical(batch_tags,len(all_tags)).reshape(batch_tags.shape+(-1,))
            yield batch_words,batch_tags_1hot
        

__Callbacks:__ Another thing we need is to measure model performance. The tricky part is not to count accuracy after sentence ends (on padding) and making sure we count all the validation data exactly once.

While it isn't impossible to persuade Keras to do all of that, we may as well write our own callback that does that.
Keras callbacks allow you to write a custom code to be ran once every epoch or every minibatch. We'll define one via LambdaCallback

__Callbacks:__ Еще нам нужно измерить производительность модели. Сложность состоит в том, чтобы не подсчитывать точность после окончания предложения (при заполнении) и не просчитывать все проверочные данные ровно один раз.

Хотя убедить Кераса сделать все это не невозможно, мы также можем написать собственный обратный вызов, который сделает это.
Обратные вызовы Keras позволяют вам написать собственный код, который будет запускаться один раз в каждую эпоху или каждый мини-пакет. Мы определим один через LambdaCallback 

In [10]:
def compute_test_accuracy(model):
    test_words,test_tags = zip(*[zip(*sentence) for sentence in test_data])
    test_words,test_tags = to_matrix(test_words, word_to_id), to_matrix(test_tags, tag_to_id)

    #predict tag probabilities of shape [batch, time, n_tags]
    predicted_tag_probabilities = model.predict(test_words, verbose=1)
    predicted_tags = predicted_tag_probabilities.argmax(axis=-1)

    #compute accurary excluding padding
    numerator = np.sum(np.logical_and((predicted_tags == test_tags), (test_words != 0)))
    denominator = np.sum(test_words != 0)
    return float(numerator)/denominator


class EvaluateAccuracy(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        sys.stdout.flush()
        print("\nMeasuring validation accuracy...")
        acc = compute_test_accuracy(self.model)
        print("\nValidation accuracy: %.5f\n"%acc)
        sys.stdout.flush()
        

In [ ]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data), len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5
1344/1343 [==============================] - 23s 17ms/step - loss: 0.2484

Measuring validation accuracy...
14335/14335 [==============================] - 4s 301us/step

Validation accuracy: 0.94049

Epoch 2/5
1344/1343 [==============================] - 23s 17ms/step - loss: 0.0580

Measuring validation accuracy...
14335/14335 [==============================] - 4s 288us/step

Validation accuracy: 0.94416

Epoch 3/5
1344/1343 [==============================] - 23s 17ms/step - loss: 0.0512

Measuring validation accuracy...
14335/14335 [==============================] - 4s 290us/step

Validation accuracy: 0.94551

Epoch 4/5
1344/1343 [==============================] - 23s 17ms/step - loss: 0.0467

Measuring validation accuracy...
14335/14335 [==============================] - 4s 291us/step

Validation accuracy: 0.94499

Epoch 5/5
1344/1343 [==============================] - 23s 17ms/step - loss: 0.0427

Measuring validation accuracy...
14335/14335 [=============================

Measure final accuracy on the whole test set.
Измерьте конечную точность на всем испытательном наборе. 

In [ ]:
acc = compute_test_accuracy(model)
print("Final accuracy: %.5f"%acc)

assert acc>0.94, "Keras has gone on a rampage again, please contact course staff."

14335/14335 [==============================] - 4s 290us/step
Final accuracy: 0.94393


### Going bidirectional

Since we're analyzing a full sequence, it's legal for us to look into future data.

A simple way to achieve that is to go both directions at once, making a __bidirectional RNN__.

In Keras you can achieve that both manually (using two LSTMs and Concatenate) and by using __`keras.layers.Bidirectional`__. 

This one works just as `TimeDistributed` we saw before: you wrap it around a recurrent layer (SimpleRNN now and LSTM/GRU later) and it actually creates two layers under the hood.

Your first task is to use such a layer our POS-tagger.

### Двунаправленный

Поскольку мы анализируем полную последовательность, нам разрешено заглядывать в будущие данные.

Простой способ добиться этого - пойти в обоих направлениях одновременно, создав __bidirectional RNN__.

В Keras вы можете добиться этого как вручную (используя два LSTM и Concatenate), так и используя __`keras.layers.Bidirectional`__.

Он работает так же, как TimeDistributed, который мы видели раньше: вы обертываете его вокруг повторяющегося слоя (SimpleRNN сейчас и LSTM / GRU позже), и он фактически создает два слоя под капотом.

Ваша первая задача - использовать такой слой в нашем POS-tagger. 

In [51]:
#Define a model that utilizes bidirectional SimpleRNN
model = keras.models.Sequential()


model.add(L.InputLayer([None], dtype='int32'))
model.add(L.Embedding(len(all_words), 50))
model.add(L.Bidirectional(L.SimpleRNN(64, return_sequences=True)))

#add top layer that predicts tag probabilities
stepwise_dense = L.Dense(len(all_tags), activation='softmax')
#stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)


In [52]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5
1344/1343 [==============================] - 69s 51ms/step - loss: 0.1919

Measuring validation accuracy...
14335/14335 [==============================] - 12s 837us/step

Validation accuracy: 0.95551

Epoch 2/5
1344/1343 [==============================] - 66s 49ms/step - loss: 0.0424

Measuring validation accuracy...
14335/14335 [==============================] - 11s 767us/step

Validation accuracy: 0.96078

Epoch 3/5
1344/1343 [==============================] - 67s 50ms/step - loss: 0.0347

Measuring validation accuracy...
14335/14335 [==============================] - 11s 779us/step

Validation accuracy: 0.96213

Epoch 4/5
1344/1343 [==============================] - 67s 50ms/step - loss: 0.0295

Measuring validation accuracy...
14335/14335 [==============================] - 11s 776us/step

Validation accuracy: 0.96266

Epoch 5/5
1344/1343 [==============================] - 67s 50ms/step - loss: 0.0252

Measuring validation accuracy...
14335/14335 [=========================

In [ ]:
acc = compute_test_accuracy(model)
print("\nFinal accuracy: %.5f"%acc)

assert acc>0.96, "Bidirectional RNNs are better than this!"
print("Well done!")

14335/14335 [==============================] - 8s 534us/step

Final accuracy: 0.96139
Well done!


**Не делаем**
Task I: The great battle between neural networks vs everything else (bonus points)

You guesses it. We're now gonna ask you to apply your neural network to the the POS tagging problem from `./data`.

In the minimal example, you can use words as the only input for the model. You can also try using the same features you used for statistical tagger - just concatenate them with word embedding.

There's also a number of things you can try.


### Задача I: великая битва между нейронными сетями и всем остальным (бонусные баллы) 
Вы уже догадались. Теперь мы попросим вас применить вашу нейронную сеть к проблеме тегов POS из ./data.

В минимальном примере вы можете использовать слова как единственный вход для модели. Вы также можете попробовать использовать те же функции, которые вы использовали для статистических тегов, - просто объедините их с помощью встраивания слов.

Есть также ряд вещей, которые вы можете попробовать. 

### Task II: Structured loss functions (more bonus points)

Since we're tagging the whole sequence at once, we might as well train our network to do so. Remember linear CRF from the lecture? You can also use it as a loss function for your RNN


  * There's more than one way to do so, but we'd recommend starting with [Conditional Random Fields](http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/)
  * You can plug CRF as a loss function and still train by backprop. There's even some neat tensorflow [implementation](https://www.tensorflow.org/api_guides/python/contrib.crf) for you.
  * Alternatively, you can condition your model on previous tags (make it autoregressive) and perform __beam search__ over that model.

### Задача II: Структурированные функции потерь (больше бонусных баллов)

Поскольку мы маркируем сразу всю последовательность, мы могли бы также обучить нашу сеть этому. Помните линейный CRF из лекции? Вы также можете использовать его как функцию потерь для своей RNN.


   * Есть несколько способов сделать это, но мы рекомендуем начать с [Условные случайные поля] (http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/)
   * Вы можете подключить CRF как функцию потерь и по-прежнему тренироваться по обратному каналу Есть даже интересная [реализация] тензорного потока (https://www.tensorflow.org/api_guides/python/contrib.crf) для вас.
   * В качестве альтернативы вы можете обусловить свою модель предыдущими тегами (сделать ее авторегрессионной) и выполнить поиск __beam__ по этой модели. 

In [11]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-yc24brc_
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-yc24brc_
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp37-none-any.whl size=101065 sha256=eb01ce23b2c95e5b93ca85fd61f118029a61624ee4ab4aaa09fe3d4bddfad307
  Stored in directory: /tmp/pip-ephem-wheel-cache-xvv1g0r0/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [48]:

from keras.layers import Embedding, Input
from keras.models import Model
#from keras_crf import CRF
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_marginal_accuracy
from keras_contrib.metrics import crf_viterbi_accuracy

model = keras.models.Sequential()
model.add(L.InputLayer([None], dtype='int32'))
model.add(L.Embedding(len(all_words), 50))
model.add(L.Bidirectional(L.SimpleRNN(64, return_sequences=True)))

# #add top layer that predicts tag probabilities
#stepwise_dense = L.Dense(len(all_tags), activation='softmax')
#stepwise_dense = L.TimeDistributed(stepwise_dense)
#model.add(stepwise_dense)

crf = CRF(units=len(all_tags), sparse_target=False)
model.add(crf)
model.summary()
model.compile('adam', loss=crf_loss)


Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, None, 50)          500100    
_________________________________________________________________
bidirectional_9 (Bidirection (None, None, 128)         14720     
_________________________________________________________________
crf_21 (CRF)                 (None, None, 14)          2030      
Total params: 516,850
Trainable params: 516,850
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.fit_generator(generate_batches(train_data), len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5)

Epoch 1/5
1344/1343 [==============================] - 119s 89ms/step - loss: 0.1918

Measuring validation accuracy...
14335/14335 [==============================] - 53s 4ms/step

Validation accuracy: 0.95663

Epoch 2/5
1344/1343 [==============================] - 119s 89ms/step - loss: 0.0366

Measuring validation accuracy...
14335/14335 [==============================] - 53s 4ms/step

Validation accuracy: 0.96168

Epoch 3/5
1344/1343 [==============================] - 120s 89ms/step - loss: 0.0094

Measuring validation accuracy...
14335/14335 [==============================] - 53s 4ms/step

Validation accuracy: 0.96270

Epoch 4/5
1344/1343 [==============================] - 119s 88ms/step - loss: -0.0188

Measuring validation accuracy...
14335/14335 [==============================] - 52s 4ms/step

Validation accuracy: 0.96212

Epoch 5/5
1344/1343 [==============================] - 118s 88ms/step - loss: -0.0476

Measuring validation accuracy...
14335/14335 [==========================

In [50]:
acc = compute_test_accuracy(model)
print("\nFinal accuracy: %.5f"%acc)

assert acc>0.96, "Bidirectional RNNs are better than this!"
print("Well done!")

14335/14335 [==============================] - 52s 4ms/step

Final accuracy: 0.96145
Well done!


Интересно, что с 4-й эпохи loss стал отрицательный и accuracy снижается

#### Some tips
Here there are a few more tips on how to improve training that are a bit trickier to impliment. We strongly suggest that you try them _after_ you've got a good initial model.
* __Use pre-trained embeddings__: you can use pre-trained weights from [there](http://ahogrammer.com/2017/01/20/the-list-of-pretrained-word-embeddings/) to kickstart your Embedding layer.
  * Embedding layer has a matrix W (layer.W) which contains word embeddings for each word in the dictionary. You can just overwrite them with tf.assign.
  * When using pre-trained embeddings, pay attention to the fact that model's dictionary is different from your own.
  * You may want to switch trainable=False for embedding layer in first few epochs as in regular fine-tuning.  
* __Go beyond SimpleRNN__: there's `keras.layers.LSTM` and `keras.layers.GRU`
  * If you want to use a custom recurrent Cell, read [this](https://keras.io/layers/recurrent/#rnn)
  * You can also use 1D Convolutions (`keras.layers.Conv1D`). They are often as good as recurrent layers but with less overfitting.
* __Stack more layers__: if there is a common motif to this course it's about stacking layers
  * You can just add recurrent and 1dconv layers on top of one another and keras will understand it
  * Just remember that bigger networks may need more epochs to train
* __Regularization__: you can apply dropouts as usual but also in an RNN-specific way
  * `keras.layers.Dropout` works inbetween RNN layers
  * Recurrent layers also have `recurrent_dropout` parameter
* __Gradient clipping__: If your training isn't as stable as you'd like, set `clipnorm` in your optimizer.
  * Which is to say, it's a good idea to watch over your loss curve at each minibatch. Try tensorboard callback or something similar.
* __Word Dropout__: tl;dr randomly replace words with UNK during training. 
  * This can also simulate increased amount of unknown words in the test set
* __Larger vocabulary__: You can obtain greater performance by expanding your model's input dictionary from 5000 to up to every single word!
  * Just make sure your model doesn't overfit due to so many parameters.
  * Combined with regularizers or pre-trained word-vectors this could be really good cuz right now our model is blind to >5% of words.  
* __More efficient batching__: right now TF spends a lot of time iterating over "0"s
  * This happens because batch is always padded to the length of a longest sentence
  * You can speed things up by pre-generating batches of similar lengths and feeding it with randomly chosen pre-generated batch.
  * This technically breaks the i.i.d. assumption, but it works unless you come up with some insane rnn architectures.
* __The most important advice__: don't cram in everything at once!
  * If you stuff in a lot of modiffications, some of them almost inevitably gonna be detrimental and you'll never know which of them are.
  * Try to instead go in small iterations and record experiment results to guide further search.
    
Good hunting!

#### Несколько советов
Вот еще несколько советов о том, как улучшить тренировку, которые немного сложнее реализовать. Мы настоятельно рекомендуем вам попробовать их _после__ того, как у вас будет хорошая начальная модель.
* __Используйте предварительно обученные вложения __: вы можете использовать предварительно обученные веса из [там] (http://ahogrammer.com/2017/01/20/the-list-of-pretrained-word-embeddings/), чтобы начать внедрение. слой.
  * Встраиваемый слой имеет матрицу W (layer.W), которая содержит вложения слов для каждого слова в словаре. Вы можете просто перезаписать их с помощью tf.assign.
  * При использовании предварительно обученных эмбеддингов обратите внимание на то, что словарь модели отличается от вашего собственного.
  * Вы можете переключить trainable = False для встраивания слоя в первые несколько эпох, как при обычной тонкой настройке.
* __Войдите за пределы SimpleRNN__: есть `keras.layers.LSTM` и` keras.layers.GRU`
  * Если вы хотите использовать настраиваемую повторяющуюся ячейку, прочтите [это] (https://keras.io/layers/recurrent/#rnn)
  * Вы также можете использовать одномерные свертки (`keras.layers.Conv1D`). Часто они не уступают повторяющимся слоям, но с меньшим количеством переобучений.
* __Складывайте больше слоев__: если у этого курса есть общий мотив, то он о наложении слоев
  * Вы можете просто добавить повторяющиеся слои и слои 1dconv друг на друга, и keras поймет это
  * Просто помните, что для более крупных сетей может потребоваться больше эпох для обучения
* __Regularization__: отсев можно применять как обычно, но также и специфичным для RNN способом.
  * `keras.layers.Dropout` работает между слоями RNN
  * Рекуррентные слои также имеют параметр recurrent_dropout
* __Gradient clipping__: Если ваше обучение не так стабильно, как хотелось бы, установите clipnorm в оптимизаторе.
  * Другими словами, рекомендуется следить за кривой потерь на каждой мини-партии. Попробуйте обратный вызов tenorboard или что-то подобное.
* __Word Dropout__: tl; dr случайным образом заменяет слова на UNK во время обучения.
  * Это также может имитировать увеличенное количество неизвестных слов в тестовом наборе
* __Больше словаря__: вы можете добиться большей производительности, расширив входной словарь вашей модели с 5000 до каждого отдельного слова!
  * Просто убедитесь, что ваша модель не переоснащается из-за такого количества параметров.
  * В сочетании с регуляризаторами или предварительно обученными векторами слов это может быть действительно хорошо, потому что сейчас наша модель слепа к> 5% слов.
* __Более эффективное пакетирование__: сейчас TF тратит много времени на итерацию по "0" сек.
  * Это происходит потому, что партия всегда дополняется до длины самого длинного предложения.
  * Вы можете ускорить процесс, предварительно сгенерировав пакеты одинаковой длины и загрузив их случайно выбранными предварительно созданными пакетами.
  * Это технически нарушает i.i.d. предположение, но это работает, если вы не придумаете какие-то безумные архитектуры rnn.
* __Самый главный совет__: не впихивайте все сразу!
  * Если вы добавите много модификаций, некоторые из них почти неизбежно будут вредными, и вы никогда не узнаете, какие из них.
  * Вместо этого попробуйте проводить небольшие итерации и записывать результаты экспериментов, чтобы направлять дальнейший поиск.
    
Удачной охоты! 